# My first notebook

> Note: to avoid random Lorem ipsum text, we've taken the step of asking chatGPT to explain the tools. It is therefore possible that there may be some oddities.

## Our pipeline

1. Download data
2. Create a genome index
3. Check quality
4. Align data with reference genome
5. Count number of read per gene
6. Diferential analysis
7. Visualisation 

## Load tools

In [ ]:
module load zenodo_get/1.3.2 
module load hisat2/2.2.1
module load fastqc/0.12.1
module load samtools/1.15.1
module load htseq/0.13.5
module load r/4.2.3

## Get Data from Zenodo

Zenodo is an online platform that enables researchers and scientists to deposit, share, and archive their research data, publications, software, presentations, and other digital content. It is an initiative developed by the European Organization for Nuclear Research (CERN) to promote the free circulation of scientific knowledge and facilitate access to research findings.

Zenodo provides free and permanent storage space for researchers, ensuring the longevity and availability of research data in the long term. Additionally, the platform assigns Digital Object Identifiers (DOIs) to each deposit, making it easier to cite and reference research works, thus contributing to the academic recognition of authors.

### Get data from Zenodo

In [ ]:
zenodo_get -t 60 -R 3 10.5281/zenodo.3997237

### Checking the viability of data 

In [ ]:
md5sum -c md5sums.txt

### Extract data

In [ ]:
tar -xzf FAIR_Bioinfo_data.tar.gz && rm FAIR_Bioinfo_data.tar.gz

## Create hisat2 indexes

### The tool used : `hisat2-build`

The `hisat2-build` command is used in bioinformatics to index a reference genome for efficient alignment of sequence reads. Here's what it does:

1. **Reference Genome Indexing:** Takes a reference genome, divides it into small parts, and creates an index.
   
2. **Partitioning into Small Parts:** Divides the reference genome into fragments of a certain size.

3. **Creating an Index:** Generates an index, which acts as a map or guide for quick retrieval of different genome parts. This index includes information about the positions of unique sequences in the genome.

4. **Optimizing Search:** The created index speeds up the alignment process by allowing quick location of relevant genome regions, saving time especially when analyzing large datasets.

In summary, `hisat2-build` creates an index from a reference genome, facilitating rapid alignment of sequence data during genetic analysis.

#### Code

In [ ]:
mkdir -p hisat2_indexes
hisat2-build Data/O.tauri_genome.fna hisat2_indexes/Otauri

## Check quality of the data

### The tool used : `fastqc`

The `fastqc` command is used in bioinformatics for quality control analysis of high-throughput sequencing data. Here's a brief overview of its functionality:

1. **Quality Control Analysis:** FastQC analyzes the quality of sequencing data to identify potential issues or biases that may affect downstream analysis.

2. **Input Data:** Takes input files in various formats (e.g., FASTQ) containing sequencing reads generated from experiments like RNA-seq, ChIP-seq, or whole-genome sequencing.

3. **Analysis Modules:** FastQC performs various analysis modules to assess different aspects of data quality, including per-base sequence quality, per-sequence quality scores, sequence length distribution, GC content, sequence duplication levels, and overrepresented sequences.

4. **Visual Reports:** Generates HTML reports containing summary statistics, graphs, and charts to visualize the quality metrics and identify any anomalies or patterns in the data.

5. **Interpretation:** Users can interpret the FastQC reports to make informed decisions about data preprocessing steps, such as trimming adapters, filtering low-quality reads, or adjusting sequencing parameters.

In summary, `fastqc` is a versatile tool for assessing the quality of sequencing data, providing valuable insights to researchers for optimizing experimental protocols and ensuring the reliability of downstream analyses.

### Code

In [ ]:
mkdir -p quality
fastqc -o quality Data/*.fastq.gz

## Align data to reference genome

### The tools we use : `hisat2` & `samtools`

#### Hisat2

`hisat2` is a bioinformatics tool commonly used for aligning high-throughput sequencing reads to a reference genome. Here's a concise overview of its functionality:

1. **Alignment of Sequencing Reads:** Hisat2 aligns sequencing reads, typically obtained from RNA-seq, DNA-seq, or other sequencing experiments, to a reference genome or transcriptome.

2. **Input Data:** Accepts input reads in FASTQ format, which contain short sequences of nucleotides generated by sequencing machines.

3. **Reference Genome Alignment:** Aligns reads to a reference genome or transcriptome to determine their genomic origin or mapping position.

4. **Sensitive and Fast Alignment:** Hisat2 employs advanced algorithms to achieve highly sensitive and accurate alignments while maintaining computational efficiency, making it suitable for analyzing large-scale sequencing datasets.

5. **Spliced Alignment:** Capable of accurately aligning reads across splice junctions in eukaryotic genomes, facilitating gene expression analysis from RNA-seq data.

6. **Output Formats:** Generates alignment results in standard formats such as SAM (Sequence Alignment/Map) or BAM (Binary Alignment/Map), which can be further processed or visualized using various bioinformatics tools.

7. **Compatibility:** Hisat2 is compatible with downstream analysis tools and pipelines commonly used in bioinformatics, allowing seamless integration into data analysis workflows.

In summary, `hisat2` is a versatile and efficient tool for aligning sequencing reads to reference genomes or transcriptomes, providing essential information for various genomic and transcriptomic analyses.

#### samtools

`Samtools` is a versatile bioinformatics tool widely used for manipulating and analyzing files in the SAM (Sequence Alignment/Map) and BAM (Binary Alignment/Map) formats, which are commonly used to represent sequence alignments generated by various sequencing platforms. Here's a concise overview of its functionality:

1. **File Format Conversion:** Samtools provides utilities for converting between SAM and BAM formats, enabling efficient storage and manipulation of alignment data.

2. **Alignment Inspection:** Allows users to view and extract alignment information from SAM and BAM files, including read sequences, mapping quality scores, alignment positions, and alignment flags.

3. **Sorting and Indexing:** Provides functions to sort alignment files by genomic coordinates and create index files for rapid retrieval of specific genomic regions, improving the efficiency of downstream analysis tasks.

4. **Alignment Filtering:** Enables users to filter alignments based on various criteria such as mapping quality, read length, alignment flags, and alignment position, allowing for the selection of high-quality alignments for further analysis.

5. **Pileup Generation:** Computes pileup information, summarizing the depth and variation of aligned reads at each genomic position, which is useful for variant calling and other genomic analyses.

6. **Variant Calling:** Supports variant calling from aligned reads, providing tools for identifying single nucleotide polymorphisms (SNPs), insertions, deletions, and structural variants in genomic data.

7. **Integration with Workflows:** Samtools can be seamlessly integrated into bioinformatics workflows, serving as a fundamental component for processing and analyzing high-throughput sequencing data.

In summary, `samtools` is a powerful toolset for manipulating and analyzing sequence alignment data, providing essential functionalities for a wide range of bioinformatics applications.

### Code 

In [ ]:
mkdir -p hisat2
for fq in Data/*.fastq.gz ; do
    echo ${fq} 
    libname=$(basename $fq .fastq.gz)
    hisat2 -x hisat2_indexes/Otauri -q -U ${fq} -S hisat2/${libname}.sam
    samtools view -b -o hisat2/${libname}.bam hisat2/${libname}.sam
    samtools sort -o hisat2/${libname}-sort.bam hisat2/${libname}.bam
    samtools index hisat2/${libname}-sort.bam
done

## Count number of reads per gene

### The tool we use : `htseq-count`

The `htseq-count` tool is used in bioinformatics for quantifying gene expression from RNA sequencing (RNA-seq) data. Here's a brief overview of its functionality:

1. **Counting Reads:** `htseq-count` counts the number of sequencing reads that align to features such as genes or exons in a reference genome.

2. **Input Data:** Takes as input aligned reads in SAM (Sequence Alignment/Map) or BAM (Binary Alignment/Map) format, typically generated by aligners like HISAT2 or STAR.

3. **Feature Annotation:** Requires a file containing annotations of genomic features, such as gene coordinates in GTF (Gene Transfer Format) or GFF (General Feature Format).

4. **Counting Strategy:** Implements a strategy to assign aligned reads to features based on their alignment coordinates. Reads overlapping multiple features or features with overlapping coordinates can be handled according to specified preferences.

5. **Output:** Produces a table or file containing the count of reads aligned to each feature, which can be used for downstream differential expression analysis or other gene expression studies.

6. **Compatible with Downstream Tools:** The output of `htseq-count` is compatible with various statistical tools and workflows used for differential expression analysis, such as DESeq2 or edgeR.

In summary, `htseq-count` is a valuable tool for quantifying gene expression from RNA-seq data, providing researchers with essential data for understanding the transcriptional landscape of biological samples.

### Code


In [ ]:
pwd
cd My1rstnotebook
pwd

In [ ]:

htseq-count -f bam r pos -s no -t gene -i ID -m intersection-nonempty hisat2/*-sort.bam Data/O.tauri_annotation.gff > counts.txt

In [ ]:
head counts.txt

## Differential analysis

### The tool we use : `Desq2`

DESeq2 is a software package for differential gene expression analysis from RNA sequencing (RNA-seq) data. It is widely used in molecular biology and genomics to identify genes whose expression varies significantly between different experimental conditions.

### Code 

In [ ]:
jupyter nbconvert --to script Deseq2.ipynb

In [ ]:
R < Deseq2.r --no-save  

![](deseq2_demo.png)